# COMPUTE LU - AI for Medicine and Life Science
### Lab: Image Classification for the HAM10000 dataset

# Instructions

This lab is about using transfer learning to learn to predict on the HAM10000 dataset. We have picked two models VGG16 and ResNet50 - you can of course choose others.

You can run this notebook locally if you wish, but a GPU is preferred.

You will need to implement some code, these are marked with comments:

```py
# your code here
```

### Goal
 
  * Train one VGG16 and one ResNet50 model that achieves above 50% accuracy.


## HAM10000
The HAM10000 dataset is a large collection of multi-source dermatoscopic images of common pigmented skin lesions

Tschandl, Philipp, 2018, "The HAM10000 dataset, a large collection of multi-source dermatoscopic images of common pigmented skin lesions", https://doi.org/10.7910/DVN/DBW86T, Harvard Dataverse, V3, UNF:6:/APKSsDGVDhwPBWzsStU5A== [fileUNF]

More information: https://www.nature.com/articles/sdata2018161

This dataset contains images of skin lesions with an associated class that represents the type of skin lesion:

 * **akiec** - Actinic keratoses and intraepithelial carcinoma / Bowen's disease, 
 * **bcc** - basal cell carcinoma,
 * **blk** - benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses), 
 * **df** - dermatofibroma, 
 * **mel** - melanoma, 
 * **nv** - melanocytic nevi
 * **vasc** - vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage).

# 0. Imports and packages

CoLab requires

In [ ]:
!pip install -U scikit-learn imbalanced-learn

All other packages are up to date in the CoLab environemtn

In [ ]:
# numeric operations
import numpy as np

# reading csv
import pandas as pd

# machine learning
import tensorflow as tf 
from tensorflow.keras import layers, optimizers, losses, applications, callbacks, Model, Sequential

# oversampling
from imblearn.over_sampling import RandomOverSampler

# plotting utillities
import matplotlib.pylab as plt

# progress
from tqdm import tqdm

# notebook utility
from IPython.display import display

# evaluation
from sklearn.metrics import classification_report, PrecisionRecallDisplay, confusion_matrix

# simple counter
from collections import Counter

# 1. Download and unpack data

In [ ]:
!wget https://fileadmin.cs.lth.se/dataset/HAM10000-compute-lu-lab.tar.gz
!tar -xvzf HAM10000-compute-lu-lab.tar.gz

# 2. Load dataset

Read metadata which is a csv with comma as the seperator, the first line is the header

In [ ]:
metadata = pd.read_csv("HAM10000_dataset/metadata.csv", sep=',', header=0)

In [ ]:
label_categorical = pd.Categorical(metadata["dx"], ordered=True)
id2label = dict( enumerate(label_categorical.categories ) )
labels = label_categorical.categories.to_list()

metadata["label"] = label_categorical.codes

In [ ]:
metadata

Shuffle the lesions by lesion_id to make sure that different images of the lesion does not appear in train and test

In [ ]:
shuffled_lesions = pd.Series(metadata["lesion_id"].unique(), name="lesion_id").sample(frac=1, random_state=1667)

In [ ]:
shuffled_lesions

## 2.1 Create splits

We will pick a 60/20/20 split.

In [ ]:
train_start, val_start, test_start = 0, int(len(shuffled_lesions)*0.6), int(len(shuffled_lesions)*0.8)

train_metadata_split = metadata[metadata["lesion_id"].isin(shuffled_lesions[0:val_start])]
val_metadata_split = metadata[metadata["lesion_id"].isin(shuffled_lesions[val_start:test_start])]
test_metadata_split = metadata[metadata["lesion_id"].isin(shuffled_lesions[test_start:])]

## 2.2 What is the distribution of classes?
Number of unique entries by label

In [ ]:
label_distribution = train_metadata_split.groupby("label").nunique()["image_id"]
label_distribution / label_distribution.sum()

From above we can se that classes 0, 3 and 6 fall below the 5% mark, which is typically a rule of thumb of when classes coud be ignored by the classifier

In [ ]:
train_metadata_split.groupby("label").nunique()

In [ ]:
val_metadata_split.groupby("label").nunique()

In [ ]:
test_metadata_split.groupby("label").nunique()

**The data is very imbalanced**

## 2.3 Trying to mitigate the imbalance
We randomly oversample the minority classes so that we have an even distribution of images.
RandomOverSampler picks the same image multiple times but in a random order.

The idea to use image ids is because these do not take much memory, we can stream in the same image multiple times instead.

In [ ]:
ros = RandomOverSampler(random_state=1667)
train_image_ids, train_labels = ros.fit_resample(train_metadata_split["image_id"].to_numpy().reshape(-1,1), train_metadata_split["label"].to_numpy())

In [ ]:
for label, count in sorted(Counter(train_labels).items()):
    print(f"{labels[label]} = {count}")
    

**Perfectly balanced** - maybe not ideal, you can change this outcome by giving a number of images per class to RandomOverSampler

## 2.4 Create tensorflow datasets

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((train_image_ids.flatten(), train_labels))
ds_val = tf.data.Dataset.from_tensor_slices((val_metadata_split["image_id"].to_numpy(), val_metadata_split["label"].to_numpy()))
ds_test = tf.data.Dataset.from_tensor_slices((test_metadata_split["image_id"].to_numpy(), test_metadata_split["label"].to_numpy()))

## 2.5 Create the loading function

Pseudo code:
 * Read the image
 * Decode the raw data as jpeg
 * Convert to float32 using `tf.cast`
 * Resize to 224, 224
 * Return image and label as a tuple

In [ ]:
# This will be compiled by tensorflow into the graph

@tf.function()
def prepare(imagename, label):
    
    # your code here

    return (None,None)  # your code here

## 2.6 Test the loading function

In [ ]:
fig = plt.figure(figsize=(16,16))
for i, (X, y) in enumerate(ds_train.shuffle(10000, seed=1667).map(prepare).take(16).as_numpy_iterator()):
    sub = fig.add_subplot(4,4, i+1)
    sub.title.set_text("Label " + id2label[y])
    sub.imshow(X/255.0)

## Benchmark the data loader
This also verify that all required images loads

In [ ]:
%%time
num_images = 0
for X,y in tqdm(ds_train.shuffle(30000).map(prepare, num_parallel_calls=tf.data.AUTOTUNE).as_numpy_iterator()):
    num_images += 1

print(num_images)

# 3. Image classification with a pretrained VGG16

The HAM10000 dataset is a realistc but hard to get good predictions for.

## 3.1 Create dataset iterators

We shuffle training by imageid to save memory and we do not cache it - we stream it in   
Valdigation and test are cached for speed

In [ ]:
train = ds_train.shuffle(30000).map(prepare, num_parallel_calls=tf.data.AUTOTUNE)
val  = ds_val.map(prepare, num_parallel_calls=tf.data.AUTOTUNE).cache().shuffle(10000)
test = ds_test.map(prepare, num_parallel_calls=tf.data.AUTOTUNE).cache()

## 3.2 Define the model

Create a network with the following suggested aritecture / try another if you like:
 
  * Input of 224x224x3
  * Pretrained VGG16 - freeze the model, no finetuning
     + Do not forget preprocessing the input using VGG16.preprocess_input
  * Flatten its output
  * Choose a number of hidden layers before the final prediction
  * final classification layer - 7 labels, with softmax activation

Regarding layer choices, you may get some inspiration from here:
https://towardsdatascience.com/illustrated-10-cnn-architectures-95d78ace614d

#### Examples
The live coding session:
https://github.com/COMPUTE-LU/AI4MedLife_imaging_2021/blob/main/slides/Ai4MedLife_imaging_day2_3/LiveCoding.ipynb

#### Documentation
*transfer models* - https://keras.io/api/applications/   
*layers* - https://keras.io/api/layers/   

In [ ]:
def create_vgg16_model():
    # your code here
    
    return None # your code here

Train it with RMSprop with an inital learning rate of 3e-5

In [ ]:
model_vgg16 = create_vgg16_model()
model_vgg16.compile(optimizer=optimizers.RMSprop(3e-5), loss=losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
model_vgg16.summary()

Observe the number of trainable parameters, there should be a large number of non-trainable parameters because the vgg16 is supposed to be frozen/not trainable.

## 3.3 Train the model for 10 epochs

`train` is a `tf.data.Dataset`, use its methods to take 8000 images, batch them into 32 images at a time and as a final operation in the chain: prefetch 8 for performance.

Use Model Checkpointing to save the best model time.  
Run the full 10 epochs. Load the best model after training.
 
*Hint:* ModelCheckpoint is constructed and added to the callbacks list of Model.fit

#### Documentation
`tf.data.Dataset` - https://www.tensorflow.org/api_docs/python/tf/data/Dataset   
`Model.fit` - https://keras.io/api/models/model_training_apis/#fit-method   
`ModelCheckpoint` - https://keras.io/api/callbacks/model_checkpoint/   

In [ ]:
# your code here

history_vgg16_ep10 = None # your code here: history as returned by `Model.fit`
model_vgg16_best = None # your code here: replace with your best model after training

## 3.3 Evaluate

 * Plot loss curve
 * Compute confusion matrix
 * Compute f1 score 
 * Plot precision/recall curves

In [ ]:
def evaluate_model(model, testset, labels):
    
    # predict outputs for testset
    y_pred_prob = None # your code here

    # find the best class for each prediction
    # input = (samples,classes)
    # output = (samples,), where each element is the best class
    y_pred = None # your code here

    y_true = np.array([y for X,y in testset.as_numpy_iterator()])

    print("Classification Report")
    print(classification_report(y_true, y_pred, labels=list(range(7)), target_names=labels))

    print("Confusion matrix")
    display(pd.DataFrame(confusion_matrix(y_true, y_pred), columns=labels, index=labels))

    fig = plt.figure(figsize=(16,8))
    for i in range(7):
        ax = fig.add_subplot(2,4, i+1)
        PrecisionRecallDisplay.from_predictions(y_true == i, y_pred_prob[:,i], name=labels[i], ax = ax)

    plt.show()

In [ ]:
def plot_loss_curves(loss, val_loss, title="Learning curves"):
    plt.figure(figsize=(12,6))
    plt.title(title)
    n = len(loss)
    plt.plot(range(1, n+1), loss, 'b',label="loss")
    plt.plot(range(1, n+1), val_loss, 'r', label="val_loss")
    plt.legend()
    plt.show()

In [ ]:
plot_loss_curves(history_vgg16_ep10.history["loss"], history_vgg16_ep10.history["val_loss"], "VGG16 - 10ep")

In [ ]:
evaluate_model(model_vgg16_best, test, labels)

## 3.4 Train the model longer

Continue from the previous model, run for 10 more epochs.

Set the `initial_epoch` argument of `Model.fit`

#### Documentation
`Model.fit` - https://keras.io/api/models/model_training_apis/#fit-method   


In [ ]:
# your code here

history_vgg16_longer = None # your code here: history as returned by `Model.fit`
model_vgg16_best = None # your code here: replace with your best model after training

## 3.5 Evaluate
Same as in 3.3

In [ ]:
plot_loss_curves(history_vgg16_ep10.history["loss"] + history_vgg16_longer.history["loss"], history_vgg16_ep10.history["val_loss"] + history_vgg16_longer.history["val_loss"], "VGG16 - 20ep")
evaluate_model(model_vgg16_best, test, labels)

## 3.6 Compare loss curves

In [ ]:
def plot_two_learning_curves(history1, history2, name1, name2, title="Comparing loss curves", history2_from_epoch=None):
    plt.figure(figsize=(12,6))
    plt.title(title)
    n = len(history1.history["loss"])
    plt.plot(range(1, n+1), history1.history["loss"], 'b',label=f"loss ({name1})")
    plt.plot(range(1, n+1), history1.history["val_loss"], 'r', label=f"val_loss ({name1})")

    start = 1
    if history2_from_epoch is not None:
        start = history2_from_epoch

    n = len(history2.history["loss"])
    plt.plot(range(start, n+start), history2.history["loss"], 'g',label=f"loss ({name2})")
    plt.plot(range(start, n+start), history2.history["val_loss"], 'm', label=f"val_loss ({name2})")

    plt.legend()
    plt.show()

In [ ]:
plot_two_learning_curves(history_vgg16_ep10, history_vgg16_ep20, "10ep", "20ep", "Comparing VGG16 at 10 and 20 ep", history2_from_epoch=10)

# 4. Image classification with a pretrained ResNet-50

## 4.1 Construct the model

Take inspiration from how it originally was trained:
https://towardsdatascience.com/illustrated-10-cnn-architectures-95d78ace614d

**Hint:** The last two layers (GlobalAveragePooling2D) and a dense layer   
**Hint:** Modify and adjust your solution from 3.2.

#### Documentation
*transfer models* - https://keras.io/api/applications/   
*layers* - https://keras.io/api/layers/   

In [ ]:
def create_resnet50_model():
    # your code here

    return None # your code here

## 4.2 Train and evalute the model

### Train for 20 epochs with model checkpointing and early stopping with a patience of 5

Take a look at:
https://keras.io/api/callbacks/

In [ ]:
# your code here

history_resnet50 = None # your code here: history as returned by `Model.fit`
model_resnet50_best = None # your code here: replace with your best model after training

In [ ]:
plot_loss_curves(history_resnet50.history["loss"],history_resnet50.history["val_loss"], "ResNet50")
evaluate_model(model_resnet50_best, test, labels)

# 5. Implement augmentation

## 5.1 Create a function that creates a augmentation layer with hyperparameters

Perform the following augmentation steps:
 * Random Translation
 * Random Rotation
 * Random Zoom
 * Random Flip

In [ ]:
def create_augmentation(translate=(0.1,0.1), rotation=0.25, zoom=(0.1,0.1)):
    augmentation = Sequential(name="augmentation")
    augmentation.add(layers.RandomTranslation(translate[0], translate[1]))
    augmentation.add(layers.RandomRotation(rotation))
    augmentation.add(layers.RandomZoom(zoom[0], zoom[1]))
    augmentation.add(layers.RandomFlip())
    return augmentation


## 5.2 Test it on some images

In [ ]:
augmentation = create_augmentation()

fig = plt.figure(figsize=(16,16))
for i, (X, y) in enumerate(ds_train.shuffle(10000, seed=1667).map(prepare).take(16).as_numpy_iterator()):
    sub = fig.add_subplot(4,4, i+1)
    sub.title.set_text("Label " + id2label[y])
    sub.imshow(augmentation(X)/255.0)

# 6. Train VGG16 or ResNet-50 with augmented data

## 6.1 Define model

In [ ]:
# your code here

## 6.2 Construct and train model

In [ ]:
# your code here

## 6.2 Evaluate

In [ ]:
# your code here

# 7. Expand and experiment

## Things to try

 * Change optimizer / learning rate
 * Pick other transfer learning models
 * Add Learning Rate scheduler: ReduceLROnPlateau
 * Adjust amount / Remove dropout
 * Change batch size
 * Change imbalance sampling settings
 * Change number of final layers, their widths
 * Replace GlobalAverage2D with Flatten
 * Unfreeze transfer learned layer and also fine tune
 * Implement a custom callback that logs the loss for every batch
